<a href="https://colab.research.google.com/github/ShesterG/Sentiment_Tools_Evaluation/blob/master/scratch_work/sentiment_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
!pip3 install config

In [3]:
!pip install vaderSentiment

In [4]:
import pandas as pd
import glob
import sys
sys.path.append('../../')
import config
import warnings
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report,f1_score

In [5]:
DATA_FILE_PATH = '/content/drive/MyDrive/NLPGh/'
CLEAN_DATA_FILE_NAME = 'Book8Clean.csv'
SAVE_FILE = True
TOKENIZED_DATA_FILE_NAME = 'Book8Tokenized.csv'

In [6]:
reviews = pd.read_csv(DATA_FILE_PATH + TOKENIZED_DATA_FILE_NAME)

In [7]:
pd.set_option('display.max_colwidth', None)

In [8]:
reviews.head()

,Unnamed: 0,tweet,location_x,pretweet,candidate,LRPredScore,LRConfi,Sentiment,Unnamed: 8,tokens
0,0,@NsiaAvena @NAkufoAddo They don't know anything about peace,NaN,they dont know anyth about peac,Nana,NaN,NaN,-1,NaN,"['they', 'dont', 'know', 'anyth', 'about', 'peac']"
1,4,@BarrowPresident @NAkufoAddo good to go may God bless an protect you,NaN,good may god bless protect you,Nana,NaN,NaN,1,NaN,"['good', 'may', 'god', 'bless', 'protect', 'you']"
2,10,"Mr President, receive the GRACE OF GOD ALMIGHTY to do what HE HAS ordain you for in Jesus Mighty Name Ã°Å¸â¢ï¿½@NakufoAddo https://t.co/O1cfDfhsMR","Accra, Ghana",presid receiv the grace god almighti what ha ordain you for jesu mighti name,Nana,1.0,1.0,1,NaN,"['presid', 'receiv', 'the', 'grace', 'god', 'almighti', 'what', 'ha', 'ordain', 'you', 'for', 'jesu', 'mighti', 'name']"
3,11,"@stonebwoyb @NAkufoAddo 1gad I can't believe it,u are congratulations dem de Npp party are corrupt",NaN,gad cant believ itu are congratul dem npp parti are corrupt,Nana,-1.0,1.0,-1,NaN,"['gad', 'cant', 'believ', 'itu', 'are', 'congratul', 'dem', 'npp', 'parti', 'are', 'corrupt']"
4,12,@NAkufoAddo Please may God continue to strengthen you to do more please Ã°Å¸â¢ï¿½,NaN,pleas may god continu strengthen you more pleas,Nana,-2.0,1.0,1,NaN,"['pleas', 'may', 'god', 'continu', 'strengthen', 'you', 'more', 'pleas']"


In [9]:
minimum_value = reviews['Sentiment'].value_counts().min()

In [10]:
def sample_minimum(reviews):
    return reviews.sample(minimum_value)

    

In [11]:
g = reviews.groupby('Sentiment')

In [12]:
new_df = g.apply(sample_minimum).reset_index(drop=True)
new_df.head()

,Unnamed: 0,tweet,location_x,pretweet,candidate,LRPredScore,LRConfi,Sentiment,Unnamed: 8,tokens
0,13102,"@tabi_henry @Joy997FM @NAkufoAddo @MrKwakuKyei1 Simply put, it's only the corrupt like you that believes corruptionÃ¢â¬Â¦ https://t.co/oRhIXbYJ7D","Accra ,Ghana",simpli put it onli the corrupt like you that believ corrupt,Nana,-2.0,1.0,-1,NaN,"['simpli', 'put', 'it', 'onli', 'the', 'corrupt', 'like', 'you', 'that', 'believ', 'corrupt']"
1,88389,@tv3_ghana @JDMahama You see demma faces den reaction as the sirens the make louder Ã°Å¸ËâÃ°Å¸ËâÃ°Å¸ËâÃ°Å¸ââ lowkey dem dey fear Ã¢ï¿½âÃ¯Â¸ï¿½Ã¢ï¿½âÃ¯Â¸ï¿½,In the clouds Ã°Å¸ÅÂ«Ã¯Â¸ï¿½,you see demma face den reaction the siren the make louder lowkey dem dey fear,John,-2.0,1.0,-1,NaN,"['you', 'see', 'demma', 'face', 'den', 'reaction', 'the', 'siren', 'the', 'make', 'louder', 'lowkey', 'dem', 'dey', 'fear']"
2,73347,@Bridget_Otoo @JDMahama @NJOAgyemang Tolerance? Try defending or supporting NPP next time and u will see fire fromÃ¢â¬Â¦ https://t.co/Kh99MwFLeH,London,toler tri defend support npp next time and will see fire from,John,1.0,1.0,-1,NaN,"['toler', 'tri', 'defend', 'support', 'npp', 'next', 'time', 'and', 'will', 'see', 'fire', 'from']"
3,5387,So once again @NAkufoAddo has frustrated Matin Amidu out of office.\n\nSad sad sad \n\nThe Mr. Incorruptible sold to usÃ¢â¬Â¦ https://t.co/SmzABIpkGd,"Accra, Ghana",onc again ha frustrat matin amidu out offic sad sad sad the mr incorrupt sold us,Nana,-3.0,1.0,-1,NaN,"['onc', 'again', 'ha', 'frustrat', 'matin', 'amidu', 'out', 'offic', 'sad', 'sad', 'sad', 'the', 'mr', 'incorrupt', 'sold', 'us']"
4,83036,"In fact\n\n@JDMahama main agenda is to steal, kill and destroy\n\nLets reject the desperate last min man. https://t.co/uN5vTOXmDM",NaN,fact main agenda steal kill and destroy let reject the desper last min man,John,-2.0,1.0,-1,NaN,"['fact', 'main', 'agenda', 'steal', 'kill', 'and', 'destroy', 'let', 'reject', 'the', 'desper', 'last', 'min', 'man']"


In [13]:
new_df['Sentiment'].value_counts()

 1    1000
-1    1000
Name: Sentiment, dtype: int64

### For VADERSentiment

In [14]:
analyser = SentimentIntensityAnalyzer()


In [15]:
def vader_sentiment(text):
    score = analyser.polarity_scores(text)
    if score['compound'] > 0:
        result = 1
    else:
        result = -1
    return result

### For TextBlob

In [16]:
def text_blob(text):
    score = TextBlob(text).sentiment.polarity
    if score > 0:
        result = 1
    else:
        result = -1
    return result

### Analysis and Evaluation


In [17]:
new_df['vader_sent'] = new_df['tokens'].apply(vader_sentiment)
new_df['textblob_sent'] = new_df['tokens'].apply(text_blob)


In [18]:
col = ['vader_sent','textblob_sent']
def pd_score(df,col,target):
    score = []
    for items in col:
        rate = accuracy_score(target, df[items])
        score.append(rate)
    scores = pd.Series(score, index=['VADER', 'TextBlob'])
    return scores
    

In [19]:
scores = pd_score(new_df,col,new_df['Sentiment'])

In [20]:
rate = accuracy_score(new_df['Sentiment'], new_df['vader_sent'])

In [21]:
rate

0.7695

In [22]:
rate = accuracy_score(new_df['Sentiment'], new_df['textblob_sent'])

In [23]:
rate

0.7095

In [24]:
new_df.head()

,Unnamed: 0,tweet,location_x,pretweet,candidate,LRPredScore,LRConfi,Sentiment,Unnamed: 8,tokens,vader_sent,textblob_sent
0,13102,"@tabi_henry @Joy997FM @NAkufoAddo @MrKwakuKyei1 Simply put, it's only the corrupt like you that believes corruptionÃ¢â¬Â¦ https://t.co/oRhIXbYJ7D","Accra ,Ghana",simpli put it onli the corrupt like you that believ corrupt,Nana,-2.0,1.0,-1,NaN,"['simpli', 'put', 'it', 'onli', 'the', 'corrupt', 'like', 'you', 'that', 'believ', 'corrupt']",1,-1
1,88389,@tv3_ghana @JDMahama You see demma faces den reaction as the sirens the make louder Ã°Å¸ËâÃ°Å¸ËâÃ°Å¸ËâÃ°Å¸ââ lowkey dem dey fear Ã¢ï¿½âÃ¯Â¸ï¿½Ã¢ï¿½âÃ¯Â¸ï¿½,In the clouds Ã°Å¸ÅÂ«Ã¯Â¸ï¿½,you see demma face den reaction the siren the make louder lowkey dem dey fear,John,-2.0,1.0,-1,NaN,"['you', 'see', 'demma', 'face', 'den', 'reaction', 'the', 'siren', 'the', 'make', 'louder', 'lowkey', 'dem', 'dey', 'fear']",-1,-1
2,73347,@Bridget_Otoo @JDMahama @NJOAgyemang Tolerance? Try defending or supporting NPP next time and u will see fire fromÃ¢â¬Â¦ https://t.co/Kh99MwFLeH,London,toler tri defend support npp next time and will see fire from,John,1.0,1.0,-1,NaN,"['toler', 'tri', 'defend', 'support', 'npp', 'next', 'time', 'and', 'will', 'see', 'fire', 'from']",1,-1
3,5387,So once again @NAkufoAddo has frustrated Matin Amidu out of office.\n\nSad sad sad \n\nThe Mr. Incorruptible sold to usÃ¢â¬Â¦ https://t.co/SmzABIpkGd,"Accra, Ghana",onc again ha frustrat matin amidu out offic sad sad sad the mr incorrupt sold us,Nana,-3.0,1.0,-1,NaN,"['onc', 'again', 'ha', 'frustrat', 'matin', 'amidu', 'out', 'offic', 'sad', 'sad', 'sad', 'the', 'mr', 'incorrupt', 'sold', 'us']",-1,-1
4,83036,"In fact\n\n@JDMahama main agenda is to steal, kill and destroy\n\nLets reject the desperate last min man. https://t.co/uN5vTOXmDM",NaN,fact main agenda steal kill and destroy let reject the desper last min man,John,-2.0,1.0,-1,NaN,"['fact', 'main', 'agenda', 'steal', 'kill', 'and', 'destroy', 'let', 'reject', 'the', 'desper', 'last', 'min', 'man']",-1,-1


In [25]:
confusion_matrix(new_df['Sentiment'], new_df['vader_sent'])

array([[773, 227],
       [234, 766]])

In [26]:
confusion_matrix(new_df['Sentiment'], new_df['textblob_sent'])

array([[723, 277],
       [304, 696]])

In [27]:
print(classification_report(new_df['Sentiment'], new_df['vader_sent']))

              precision    recall  f1-score   support

          -1       0.77      0.77      0.77      1000
           1       0.77      0.77      0.77      1000

    accuracy                           0.77      2000
   macro avg       0.77      0.77      0.77      2000
weighted avg       0.77      0.77      0.77      2000



In [28]:
print(classification_report(new_df['Sentiment'], new_df['textblob_sent']))

              precision    recall  f1-score   support

          -1       0.70      0.72      0.71      1000
           1       0.72      0.70      0.71      1000

    accuracy                           0.71      2000
   macro avg       0.71      0.71      0.71      2000
weighted avg       0.71      0.71      0.71      2000

